# Alice, Bob & Eve: Symmetric Cryptography with key derivation

## Alice

The start of conversation is the same as in the previous example. Alice has to generate a new secret key by grabbing enough entrophy from the chaos (and securely transfer it to Bob):

In [2]:
var rng = System.Security.Cryptography.RandomNumberGenerator.Create();
var secretKey = new byte[32]; // 32 bytes = 256 bits
rng.GetBytes(secretKey);
Console.WriteLine($"Generated Secret Key: {Convert.ToBase64String(secretKey)}");

Generated Secret Key: EQq9+S7XzgmEOHSL0+qxYX8LewopDzDesRAPxnHVEws=


But this time, Alice will not use this key to actually encrypt the message. She will use it to create a new completely original key, specific to given message.

In [7]:
// Derive a per-message key using HKDF
byte[] GetDerivedKey(int messageNumber) {
    var info = BitConverter.GetBytes(messageNumber);

    return System.Security.Cryptography.HKDF.DeriveKey(
        System.Security.Cryptography.HashAlgorithmName.SHA256, // Name of used hash
        secretKey,  // The master key
        32,         // Length of derived key in bytes
        null,       // Optional salt
        info);      // Context info - message number
}

for (int i = 1; i <= 5; i++) {
    var derivedKey = GetDerivedKey(i);
    Console.WriteLine($"Derived Key for Message {i}: {Convert.ToBase64String(derivedKey)}");
}

Derived Key for Message 1: M5DKLSRPfK24L5yAScsp38dm9dme8hzb2QFk43paKHg=
Derived Key for Message 2: QnoUXQ8qmkNx7s0ZQRJg9Xjih70fXDcRuCAXfLSS150=
Derived Key for Message 3: uH43OOl4bxzzwrZD9mBZTO2hhU2H7vjuDPWV1yEnmnk=
Derived Key for Message 4: BLfD8iBmLWKsRYhijeBOlCEN9sGN9CKMWW5qXraZQ3g=
Derived Key for Message 5: RtJo6HtBP6WelI3VEXQU82TmKkznn6GAk17CVrLvoDE=


Alice will encrypt her message as in the previous example, but she has to keep counter so she knows what the next message number will be and cannot repeat herself. Also the message data structure has to include the message number.

In [8]:
var numberOfSentMessages = 0;

string EncryptMessage(string message) {
    // Increment message counter
    numberOfSentMessages++;

    // Create AES-GCM with derived key
    var aes = new System.Security.Cryptography.AesGcm(GetDerivedKey(numberOfSentMessages), 16);

    // Generate nonce
    var nonce = new byte[12];
    rng.GetBytes(nonce);

    // Encrypt message
    var plainTextBytes = System.Text.Encoding.UTF8.GetBytes(message);
    var cipherText = new byte[plainTextBytes.Length];
    var tag = new byte[16];
    aes.Encrypt(nonce, plainTextBytes, cipherText, tag);

    // Combine nonce, tag, and cipher text
    var messageData = nonce
        .Concat(tag)
        .Concat(cipherText)
        .ToArray();

    // Prefix the message with the message number
    return numberOfSentMessages + "|" + Convert.ToBase64String(messageData);
}

var encryptedMessage1 = EncryptMessage("Hello Bob, this is Alice.");
Console.WriteLine($"Encrypted Message 1: {encryptedMessage1}");
var encryptedMessage2 = EncryptMessage("Here is another message for you.");
Console.WriteLine($"Encrypted Message 2: {encryptedMessage2}");
var encryptedMessage3 = EncryptMessage("BTW, still love you!");
Console.WriteLine($"Encrypted Message 3: {encryptedMessage3}");

Encrypted Message 1: 1|OYq6s9K7LLoSkCoCvqxUzOuFEtvH2ikQSl0LKkUCZGKr4n/R4EYqqgFhu/L8zFO5Kb0YD+4=
Encrypted Message 2: 2|51kS9QecMuB50a5WbeAUAWKyxgahAZ7siGVoTVftDH9xjcraaXxq9kwWbTxjopRIOFQCGRz1SGvKinPV
Encrypted Message 3: 3|8IiAEdCIhsiD6Att6G8wKp5r/pbG53jwNzT4Z8ozIhWoKHHseRWML1XL2N8B6/oB


## Bob

Bob's actions mirror the ones performed by Alice. He should also keep counter of received messages. Using this counter he can also detect _replay attack_, if Eve is trying to send the same message twice. He will know that he already seen that message. He can also detect that some message is missing - maybe Eve is not above using jamming, in addition to eavesdropping!

In [9]:
var numberOfReceivedMessages = 0;

string DecryptMessage(string receivedMessage) {
    // Split message number and data
    var parts = receivedMessage.Split('|', 2);
    var messageNumber = int.Parse(parts[0]);
    var messageData = Convert.FromBase64String(parts[1]);

    var expectedMessageNumber = numberOfReceivedMessages + 1;
    if(messageNumber < expectedMessageNumber) {
        Console.WriteLine($"ERROR: Replay attack detected! Received message number {messageNumber}, but expected at least {expectedMessageNumber}.");
        return string.Empty;
    }
    if(messageNumber > expectedMessageNumber) {
        Console.WriteLine($"WARNING: Out-of-order message detected! Received message number {messageNumber}, but expected {expectedMessageNumber}.");
    }

    // Increment received message counter
    numberOfReceivedMessages++;

    // Parse nonce, tag, and cipher text
    var parsedNonce = messageData.Take(12).ToArray();
    var parsedTag = messageData.Skip(12).Take(16).ToArray();
    var parsedCipherText = messageData.Skip(28).ToArray();

    // Create AES-GCM with derived key
    var aes = new System.Security.Cryptography.AesGcm(GetDerivedKey(numberOfReceivedMessages), 16);

    // Decrypt message
    var decryptedBytes = new byte[parsedCipherText.Length];
    aes.Decrypt(parsedNonce, parsedCipherText, parsedTag, decryptedBytes);
    return System.Text.Encoding.UTF8.GetString(decryptedBytes);
}

var decryptedMessage1 = DecryptMessage(encryptedMessage1);
Console.WriteLine($"Decrypted Message 1: {decryptedMessage1}");
var decryptedMessage2 = DecryptMessage(encryptedMessage2);
Console.WriteLine($"Decrypted Message 2: {decryptedMessage2}");
var decryptedMessage3 = DecryptMessage(encryptedMessage3);
Console.WriteLine($"Decrypted Message 3: {decryptedMessage3}");
var decryptedMessage4 = DecryptMessage(encryptedMessage2);
Console.WriteLine($"Decrypted Message 4: {decryptedMessage4}");

Decrypted Message 1: Hello Bob, this is Alice.
Decrypted Message 2: Here is another message for you.
Decrypted Message 3: BTW, still love you!
ERROR: Replay attack detected! Received message number 2, but expected at least 4.
Decrypted Message 4: 


Alice and Bob used symmetric key derivation to overcome the need to trasfer the fresh secret key with every message. Good them, but they still have to find out how to escape Eve's eavesdropping for the single symmetric key transport. How? I'll show you next time.